In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer


In [5]:
import torch

In [9]:
device = torch.get_default_device()
torch.set_default_device(device)

In [12]:

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [27]:
import math
import time


# prepare the model input
prompt = "I will give some job description and my relevant experience. Will you rewrite the experience so that it well suited to the job description. Also, you need to suggest if some quantitative measures can be added to more polish the experience."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)


start_time = time.time()
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

end_time = time.time()

output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

elapsed_time = end_time - start_time
elapsed_seconds = math.ceil(elapsed_time)
print(f"Elapsed Time: {elapsed_seconds // 60}:{elapsed_seconds % 60}")



thinking content: <think>
Okay, let's start by looking at the user's job description and their relevant experience. The user wants the experience rewritten to fit the job description and to suggest quantitative measures for polishing the experience. 

First, I need to compare the job description with the experience. The user mentioned that the experience is not well-suited, so I should identify what areas need improvement. For example, if the job requires data analysis skills, the experience should highlight relevant skills and projects.

Next, I should check for any gaps. Maybe the user didn't mention specific projects or achievements that align with the job requirements. Quantitative measures can help add value by showing measurable results. For instance, if the job involves project management, they could mention tasks completed, time spent, or metrics achieved.

I need to make sure the rewritten experience is concise and highlights the key points. Also, adding quantitative metrics w